In [1]:
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder

### Read train data and test data

In [2]:
df_train = pd.read_csv("train_new.csv")
print('Number of train records: {}'.format(len(df_train)))
df_train.head()

Number of train records: 60000


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent
0,2021-09,jurong east,257,Jurong East Street 24,3-room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100


In [3]:
df_test = pd.read_csv("test_new.csv")
print('Number of test records: {}'.format(len(df_test)))
df_test.head()

Number of test records: 30000


,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region
2,2023-07,clementi,708,Clementi West Street 2,4-room,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region
3,2021-08,jurong east,351,Jurong East Street 31,3-room,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region


# Stock price 

### Read stock price data

In [4]:
df_stock = pd.read_csv("auxiliary-data/sg-stock-prices.csv")
print('Number of stock records: {}'.format(len(df_stock)))
df_stock.head()

Number of stock records: 35498


,name,symbol,date,open,high,low,close,adjusted_close
0,DBS Group,D05.SI,2021-01-04,25.13,25.34,25.01,25.34,22.833
1,DBS Group,D05.SI,2021-01-05,25.22,25.35,25.09,25.35,22.842
2,DBS Group,D05.SI,2021-01-06,25.30,25.35,25.07,25.18,22.689
3,DBS Group,D05.SI,2021-01-07,25.68,26.18,25.63,26.05,23.473
4,DBS Group,D05.SI,2021-01-08,26.55,27.47,26.50,27.42,24.707


### For stock price data, "adjusted_close" is selected to evaluate the performance of market

In [5]:
df_stock = df_stock[['date', 'adjusted_close']]
df_stock['date'] = [x[0:7] for x in df_stock['date']]
df_stock.head()

,date,adjusted_close
0,2021-01,22.833
1,2021-01,22.842
2,2021-01,22.689
3,2021-01,23.473
4,2021-01,24.707


### Group by date and take average of adjust_close

In [6]:
df_stock = df_stock.groupby('date', as_index=False)['adjusted_close'].mean()
print(df_stock)

       date  adjusted_close
0   2020-12       33.827600
1   2021-01       12.332232
2   2021-02       13.822067
3   2021-03       13.250911
4   2021-04       14.085999
5   2021-05       13.856788
6   2021-06       14.493495
7   2021-07       14.274470
8   2021-08       15.318542
9   2021-09       15.448564
10  2021-10       17.422356
11  2021-11       16.817964
12  2021-12       13.473724
13  2022-01       12.276967
14  2022-02       11.915369
15  2022-03       11.224848
16  2022-04       11.072466
17  2022-05       10.278250
18  2022-06        9.630104
19  2022-07        9.392984
20  2022-08       10.081974
21  2022-09        9.344795
22  2022-10        8.983589
23  2022-11        9.137518
24  2022-12        9.329228
25  2023-01        9.668867
26  2023-02        9.731473
27  2023-03        9.487512
28  2023-04        9.781855
29  2023-05        9.449967
30  2023-06        9.464873
31  2023-07        9.276715


### Check if the date of stock price covers all months in train and test data

In [7]:
date_stock = df_stock['date'].unique()
date_train = df_train['rent_approval_date'].unique()
date_test = df_test['rent_approval_date'].unique()
print("Date in stock data covers all in train data? {}".format(all(item in date_stock for item in date_train)))
print("Date in stock data covers all in test data? {}".format(all(item in date_stock for item in date_test)))

Date in stock data covers all in train data? True
Date in stock data covers all in test data? True


### Add in new column "avg_stock_price" for train and test data

In [8]:
# add new column to train data
list_index_date = [df_stock.index[df_stock['date'] == x][0] for x in df_train['rent_approval_date']]
list_avg_stock_price = df_stock['adjusted_close'][list_index_date].to_list()
df_train['avg_stock_price'] = list_avg_stock_price
df_train.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent,avg_stock_price
0,2021-09,jurong east,257,Jurong East Street 24,3-room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600,15.448564
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250,10.278250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900,8.983589
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850,15.318542
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100,9.137518


In [9]:
# add new column to test data
list_index_date = [df_stock.index[df_stock['date'] == x][0] for x in df_test['rent_approval_date']]
list_avg_stock_price = df_stock['adjusted_close'][list_index_date].to_list()
df_test['avg_stock_price'] = list_avg_stock_price
df_test.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,avg_stock_price
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,9.668867
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,9.344795
2,2023-07,clementi,708,Clementi West Street 2,4-room,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,9.276715
3,2021-08,jurong east,351,Jurong East Street 31,3-room,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,15.318542
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,11.224848


# Data transformation of "town" column

### frequency encoding?

In [10]:
print("#count for each value of town in train data: \n{}\n".format(df_train['town'].value_counts()))
town_nom = (df_train.groupby('town').size()) / len(df_train)
print(town_nom)

#count for each value of town in train data: 
jurong west        4413
tampines           4183
sengkang           4003
bedok              3644
ang mo kio         3494
yishun             3378
bukit merah        3160
woodlands          3031
hougang            2859
punggol            2700
toa payoh          2407
clementi           2272
bukit batok        2236
choa chu kang      2117
queenstown         2033
kallang/whampoa    1860
geylang            1716
pasir ris          1712
jurong east        1710
bukit panjang      1607
sembawang          1446
bishan             1437
serangoon          1299
central             615
marine parade       612
bukit timah          56
Name: town, dtype: int64

town
ang mo kio         0.058233
bedok              0.060733
bishan             0.023950
bukit batok        0.037267
bukit merah        0.052667
bukit panjang      0.026783
bukit timah        0.000933
central            0.010250
choa chu kang      0.035283
clementi           0.037867
geylang            

### binary encoding?

In [11]:
df_train_town = BinaryEncoder(cols=['town']).fit_transform(df_train)
df_train_town.head()

,rent_approval_date,town_0,town_1,town_2,town_3,town_4,block,street_name,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,monthly_rent,avg_stock_price
0,2021-09,0,0,0,0,1,257,Jurong East Street 24,3-room,new generation,67.0,1983,1.344518,103.738630,yuhua east,jurong east,west region,1600,15.448564
1,2022-05,0,0,0,1,0,119,bedok north road,4-room,new generation,92.0,1978,1.330186,103.938717,bedok north,bedok,east region,2250,10.278250
2,2022-10,0,0,0,1,1,157,lorong 1 toa payoh,3-room,improved,67.0,1971,1.332242,103.845643,toa payoh central,toa payoh,central region,1900,8.983589
3,2021-08,0,0,1,0,0,250,Pasir Ris Street 21,executive,apartment,149.0,1993,1.370239,103.962894,pasir ris drive,pasir ris,east region,2850,15.318542
4,2022-11,0,0,1,0,1,34,Whampoa West,3-room,improved,68.0,1972,1.320502,103.863341,bendemeer,kallang,central region,2100,9.137518
